# 20220831の評価

In [1]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
%env TOKEN=*************************************
! git clone https://$$TOKEN@github.com/konnitiha3/MOD2NN.git

import sys
sys.path.append('/content/MOD2NN')

from google.colab import drive
drive.mount('/content/drive')

env: TOKEN=*************************************
Cloning into 'MOD2NN'...
remote: Enumerating objects: 1264, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1264 (delta 6), reused 11 (delta 3), pack-reused 1235
Receiving objects: 100% (1264/1264), 39.48 MiB | 10.69 MiB/s, done.
Resolving deltas: 100% (711/711), done.
Mounted at /content/drive


In [2]:
import time
import os
import json
import sys
import re
import glob
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from Faraday.two_dim.module.lib.layers import *
from Faraday.two_dim.module.lib import regularizer

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.2
Python: 3.7.14 (default, Sep  8 2022, 00:06:44) 
[GCC 7.5.0]


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
path = "/content/drive/MyDrive/D2NN/trained_model/20220831/*"
files = glob.glob(path)

In [8]:
def binary_eval(model_path):
  model = tf.keras.models.load_model(model_path)
  eval = model.evaluate(x_test, y_test)

  bi_model = tf.keras.models.load_model(model_path)
  pattern = r'mo'
  mo_layers = []
  save = False
  each_save = False
  for layer in bi_model.layers:
      result = re.match(pattern, layer.name)
      if result:
          mo_layers.append(layer)

  for layer in mo_layers:
      w = layer.get_weights()
      bi_w = np.where(w[0]>0, np.pi/2, -np.pi/2)
      w[0] = bi_w
      layer.set_weights(w)

  bi_eval = bi_model.evaluate(x_test, y_test)
  return eval, bi_eval

In [37]:
eval_dict = {}

for file_ in files:
  l1_files = glob.glob(file_ + "/*")
  p = re.compile(r'\d+_\d+')
  l1_files.sort(reverse=False, key=lambda s: int(p.search(s).group()))

  name = os.path.split(file_)[1]

  eval_list_l1 = []

  for l1_file in l1_files:
    print(l1_file)
    seed_name = os.path.split(l1_file)[1]
    eval, bi_eval = binary_eval(l1_file)
    eval_list_l1.append([seed_name] + eval + bi_eval)
  
  eval_dict[name] = eval_list_l1
  print(eval_dict)

/content/drive/MyDrive/D2NN/trained_model/20220831/l1_1.000000e+00/seed_1
313/313 [==============================] - 5s 16ms/step - loss: 2.7588 - accuracy: 0.1098
/content/drive/MyDrive/D2NN/trained_model/20220831/l1_1.000000e+00/seed_2
313/313 [==============================] - 5s 14ms/step - loss: 2.7933 - accuracy: 0.1252
/content/drive/MyDrive/D2NN/trained_model/20220831/l1_1.000000e+00/seed_3
313/313 [==============================] - 5s 15ms/step - loss: 2.6176 - accuracy: 0.1509
/content/drive/MyDrive/D2NN/trained_model/20220831/l1_1.000000e+00/seed_4
313/313 [==============================] - 5s 14ms/step - loss: 2.5993 - accuracy: 0.1219
/content/drive/MyDrive/D2NN/trained_model/20220831/l1_1.000000e+00/seed_5
313/313 [==============================] - 5s 14ms/step - loss: 2.6751 - accuracy: 0.0820
/content/drive/MyDrive/D2NN/trained_model/20220831/l1_1.000000e+00/seed_6
313/313 [==============================] - 5s 15ms/step - loss: 2.9370 - accuracy: 0.0547
/content/drive/M

In [63]:
import openpyxl
# 数値処理
import numpy as np
import pandas as pd

workbook = openpyxl.Workbook()
# ワークブックをエクセルファイルとして保存
workbook.save('sample.xlsx')

writer = pd.ExcelWriter('sample.xlsx')

for key in list(eval_dict.keys()):
  df = pd.DataFrame(eval_dict[key], columns=["seed", "loss", "acc", "bi_loss", "bi_acc"])
  df.to_excel(writer, sheet_name=key, index=False)

# エクセルファイルを保存
writer.save()

FileNotFoundError: ignored